In [1]:
facts = {} # facts we know to be true
antifacts = {} # facts we know to be false

def add_fact(name, args):
    # Add a fact that we know to be true
    print(f"adding fact ({name}, {args})")
    if name not in facts:
        facts[name] = [args]
    else:
        if args not in facts[name]:
            facts[name].append(args)

def add_antifact(name, args):
    # Add a fact that we know to be false
    print(f"adding antifact ({name}, {args})")

    if name not in antifacts:
        antifacts[name] = [args]
    else:
        if args not in antifacts[name]:
            antifacts[name].append(args)
    
def is_true(name, args):
    return name in facts and args in facts[name]

def is_false(name, args):
    return name in antifacts and args in antifacts[name]

def get_value(name, args):
    # Get three-valued value of a predicate
    if is_true(name, args): return "T"
    if is_false(name, args): return "F"
    return "U"


example = 1
if example == 1:
    # - Sample program 1 -
    # assignment("A1", 0).
    # assignment("A2", 1).
    # successor("A1", "A2").

    # - Facts we would expect to be derived from program 1 -
    # may_have_value("A1", 0)
    # may_have_value("A2", 1)
    # strong_update("A2")

    # - In particular, it should *not* derive may_have_value("A2", 0) -
    lines = ["B0", "B1", "B2"]
    vals = [0, 1, 42]
    add_fact("assignment", ["B0", 42])
    add_fact("assignment", ["B1", 0])
    add_fact("assignment", ["B2", 1])
    add_fact("successor", ["B0", "B1"])
    add_fact("successor", ["B1", "B2"])
    add_antifact("n.assignment", ["B0", 42])
    add_antifact("n.assignment", ["B1", 0])
    add_antifact("n.assignment", ["B2", 1])
    add_antifact("n.successor", ["B0", "B1"])
    add_antifact("n.successor", ["B1", "B2"])

elif example == 2:
    # - Sample program 2 -
    # assignment("B0", 42).
    # assignment("B1", 0).
    # assignment("B2", 1).
    # successor("B0", "B1").
    # successor("B1", "B2").
    
    # - Facts we would expect to be derived from program 2 -
    # may_have_value("B0", 42).
    # may_have_value("B1", 42).
    # may_have_value("B1", 0).
    # may_have_value("B2", 42).
    # may_have_value("B2", 0).
    # may_have_value("B2", 1).

    lines = ["A1", "A2"]
    vals = [0, 1, 42]
    add_fact("assignment", ["A1", 0])
    add_fact("assignment", ["A2", 1])
    add_fact("successor", ["A1", "A2"])
    add_antifact("n.assignment", ["A1", 0])
    add_antifact("n.assignment", ["A2", 1])
    add_antifact("n.successor", ["A1", "A2"])

# We know each line can only have one assignment
for l in lines:
    for v in vals:
        if not is_true("assignment", [l, v]):
            add_fact("n.assignment", [l, v])

for l1 in lines:
    for l2 in lines:
        if not is_true("successor", [l1, l2]):
            add_fact("n.successor", [l1, l2])

# Add the anti-facts corresponding to the facts we know
t = ["assignment", "n.assignment"]
for i in range(2):
    for j in facts[t[i]]:
        add_antifact(t[1-i], j)

t = ["successor", "n.successor"]
for i in range(2):
    for j in facts[t[i]]:
        add_antifact(t[1-i], j)


adding fact (assignment, ['B0', 42])
adding fact (assignment, ['B1', 0])
adding fact (assignment, ['B2', 1])
adding fact (successor, ['B0', 'B1'])
adding fact (successor, ['B1', 'B2'])
adding antifact (n.assignment, ['B0', 42])
adding antifact (n.assignment, ['B1', 0])
adding antifact (n.assignment, ['B2', 1])
adding antifact (n.successor, ['B0', 'B1'])
adding antifact (n.successor, ['B1', 'B2'])
adding fact (n.assignment, ['B0', 0])
adding fact (n.assignment, ['B0', 1])
adding fact (n.assignment, ['B1', 1])
adding fact (n.assignment, ['B1', 42])
adding fact (n.assignment, ['B2', 0])
adding fact (n.assignment, ['B2', 42])
adding fact (n.successor, ['B0', 'B0'])
adding fact (n.successor, ['B0', 'B2'])
adding fact (n.successor, ['B1', 'B0'])
adding fact (n.successor, ['B1', 'B1'])
adding fact (n.successor, ['B2', 'B0'])
adding fact (n.successor, ['B2', 'B1'])
adding fact (n.successor, ['B2', 'B2'])
adding antifact (n.assignment, ['B0', 42])
adding antifact (n.assignment, ['B1', 0])
addin

In [2]:
# AND of our 3-valued semantics
def and_l(l):
    if l == ["T"] * len(l):
        return "T"
    elif "F" in l:
        return "F"
    else:
        return "U"

# OR in our 3-valued semantics
def or_l(l):
    if l == ["F"] * len(l):
        return "F"
    elif "T" in l:
        return "T"
    else:
        return "U"

# whether the variable may have a value of v on a given line
def may_have_value(line, v):
    if is_true("may_have_value", [line, v]) or is_false("may_have_value", [line, v]):
        return "A"
    
    clause_1 = get_value("assignment", [line, v])

    clause_2_values = []
    
    for prev in lines:
        clause_2_values.append(and_l([
            get_value("successor", [prev, line]),
            get_value("may_have_value", [prev, v]),
            get_value("n.strong_update", [line])
        ]))
    clause_2 = or_l(clause_2_values)
    
    return or_l([clause_1, clause_2])

# whether or not strong update might apply at a given line
def strong_update(line):
    if is_true("strong_update", [line]) or is_false("strong_update", [line]):
        return "A"
    
    values = []
    for prev in lines:
        values.append(and_l([
            get_value("successor", [prev, line]),
            get_value("n.may_have_value", [prev, 42])
        ]))
    return or_l(values)

def n_may_have_value(line, v):
    if is_true("n.may_have_value", [line, v]) or is_false("n.may_have_value", [line, v]):
        return "A"
    
    clause_1 = get_value("n.assignment", [line, v])

    clause_2_values = []
    
    for prev in lines:
        clause_2_values.append(or_l([
            get_value("n.successor", [prev, line]),
            get_value("n.may_have_value", [prev, v]),
            get_value("strong_update", [line])
        ]))
    clause_2 = and_l(clause_2_values)
    
    return and_l([clause_1, clause_2])

def n_strong_update(line):
    if is_true("n.strong_update", [line]) or is_false("n.strong_update", [line]):
        return "A"
    
    values = []
    for prev in lines:
        values.append(or_l([
            get_value("n.successor", [prev, line]),
            get_value("may_have_value", [prev, 42])
        ]))
    return and_l(values)

changes = True
while (changes):
    changes = False
    for line in lines:
        if strong_update(line) == "T":
            changes = True
            add_fact("strong_update", [line])
        elif strong_update(line) == "F":
            changes = True
            add_antifact("strong_update", [line])

    for l in lines:
        for v in vals:
            if n_may_have_value(l, v) == "T":
                changes = True
                add_fact("n.may_have_value", [l, v])
            elif n_may_have_value(l, v) == "F":
                changes = True
                add_antifact("n.may_have_value", [l, v])

changes = True
while (changes):
    changes = False
    for line in lines:
        if n_strong_update(line) == "T":
            changes = True
            add_fact("n.strong_update", [line])
        elif n_strong_update(line) == "F":
            changes = True
            add_antifact("n.strong_update", [line])

    for l in lines:
        for v in vals:
            if may_have_value(l, v) == "T":
                changes = True
                add_fact("may_have_value", [l, v])
            elif may_have_value(l, v) == "F":
                changes = True
                add_antifact("may_have_value", [l, v])


adding antifact (strong_update, ['B0'])
adding fact (n.may_have_value, ['B0', 0])
adding fact (n.may_have_value, ['B0', 1])
adding antifact (n.may_have_value, ['B0', 42])
adding antifact (n.may_have_value, ['B1', 0])
adding fact (n.may_have_value, ['B1', 1])
adding antifact (n.may_have_value, ['B2', 1])
adding antifact (strong_update, ['B1'])
adding antifact (n.may_have_value, ['B1', 42])
adding antifact (strong_update, ['B2'])
adding antifact (n.may_have_value, ['B2', 0])
adding antifact (n.may_have_value, ['B2', 42])
adding fact (n.strong_update, ['B0'])
adding antifact (may_have_value, ['B0', 0])
adding antifact (may_have_value, ['B0', 1])
adding fact (may_have_value, ['B0', 42])
adding fact (may_have_value, ['B1', 0])
adding antifact (may_have_value, ['B1', 1])
adding fact (may_have_value, ['B2', 1])
adding fact (n.strong_update, ['B1'])
adding fact (may_have_value, ['B1', 42])
adding fact (n.strong_update, ['B2'])
adding fact (may_have_value, ['B2', 0])
adding fact (may_have_value

In [3]:
def show(predicate):
    if predicate in facts:
        print(f"{predicate}: {facts[predicate]}")
    else:
        print(f"{predicate}: {[]}")

for pred in ["may_have_value", "strong_update", "n.may_have_value", "n.strong_update"]:
    show(pred)

may_have_value: [['B0', 42], ['B1', 0], ['B2', 1], ['B1', 42], ['B2', 0], ['B2', 42]]
strong_update: []
n.may_have_value: [['B0', 0], ['B0', 1], ['B1', 1]]
n.strong_update: [['B0'], ['B1'], ['B2']]
